In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as  pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple

import utils

#Configuración de notebook
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

## PREPARAR DATAFRAME INICIAL

In [ ]:
df = utils.build_initial_dataframe("Inter Miami")

In [21]:
# df_inter_matches_events_v2.to_csv('../data/raw/inter_miami_mls24_events.csv', index=False)

In [10]:
df = pd.read_csv('../data/raw/inter_miami_mls24_events.csv')

In [12]:
df.columns

Index(['id', 'eventId', 'typeId', 'eventTypeName', 'periodId', 'timeMin',
       'timeSec', 'contestantId', 'outcome', 'x',
       ...
       'Low Right', 'Pull back', 'Big chance', 'Other body part',
       'Touch type clearance', 'Sliding', 'Take on space',
       'Small box - Centre', 'Shirt Pull/Holding', 'Follows a Dribble'],
      dtype='object', length=182)

## ANÁLISIS EXPLORATORIO

In [23]:
utils.exploratory_analysis(df)

=== Información del Dataset ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64971 entries, 0 to 64970
Data columns (total 182 columns):
 #    Column                                  Dtype  
---   ------                                  -----  
 0    id                                      int64  
 1    eventId                                 int64  
 2    typeId                                  int64  
 3    eventTypeName                           object 
 4    periodId                                int64  
 5    timeMin                                 int64  
 6    timeSec                                 int64  
 7    contestantId                            object 
 8    outcome                                 int64  
 9    x                                       float64
 10   y                                       float64
 11   timeStamp                               object 
 12   lastModified                            object 
 13   Involved                                ob

In [24]:
df.describe()

,id,eventId,typeId,periodId,timeMin,timeSec,outcome,x,y,Team Formation,Team kit,Pass End X,Pass End Y,Length,Angle,Opposite related event ID,Deleted Event Type,Defensive,Not visible,Related event ID,Foul,Goalmouth Y Coordinate,Goalmouth Z Coordinate,Blocked X Coordinate,Blocked Y Coordinate,GK X Coordinate,GK Y Coordinate,keyPass,Assist,Box - Centre,Right footed,Regular play,Assisted,Low Centre,Blocked,Intentional Assist,First Touch,Def block,Out of box - Centre,Right,Volley,Swerve Right,Not past goal line,Individual play,Mis-hit,Not assisted,Next event Goal-Kick,Free kick taken,Direct,Leading to attempt,Related error 1 ID,assist,Leading to goal,GK x coordinate time of goal,GK y coordinate time of goal,Offensive,Next event Throw-In,Minutes,Flick-on,Blocked pass,End type,Left,Event type review,Review,Out of play,Formation slot,Detailed Position ID,Position Side ID,2nd related event ID,Parried safe,Standing,Hands,Fantasy Assist Team,Fantasy assist ID,Pre-Review Event Type,Reviewed event ID,Head,Blocked cross,Viral,Video coverage lost,Long ball,Launch,Shove/Push,Box - Left,Temp Shot On,High claim,Parried danger,Diving,Chipped,Indirect,Cross,Box - Deep Left,Second (2nd) opposite related event ID,Causing player,Referee delay,Awaiting official's decision,VAR Delay,Time wasting,Head pass,Goal Kick,Low,Fast break,Lay-off,Left footed,Close High,Small box - Left,Temp Blocked,Penalty,Attempted Tackle,Second yellow,Reckless offence,Goal disallowed,Other reason,Set piece,Overrun,Corner taken,In-swinger,Hit Woodwork,Hit Right Post,Captain change,Throw in,Collected,Stooping,Box - Right,Aerial Foul,Jumping,Close Right and High,Swerve Left,Take on overtake,Caught,Temp Missed,Simulation,Unchallenged,35+ Centre,Temp Miss Not Passed Goal Line,Yellow Card,Dissent,Drinks Break,Keeper Throw,Handball,High Left,Low Right,Pull back,Big chance,Other body part,Touch type clearance,Sliding,Take on space,Small box - Centre,Shirt Pull/Holding,Follows a Dribble
count,6.497100e+04,64971.000000,64971.000000,64971.000000,64971.000000,64971.000000,64971.000000,64971.000000,64971.000000,128.000000,74.000000,40829.000000,40829.000000,40696.000000,40696.000000,13508.000000,1653.000000,165.0,22.0,1833.000000,128.000000,989.000000,989.000000,544.000000,544.000000,751.000000,751.000000,642.0,723.000000,12.0,9.0,28.0,11.0,3.0,3.0,10.0,6.0,3.0,5.0,11.0,2.0,1.0,1.0,4.0,2.0,2.0,31.0,6.0,25.0,28.000000,4.100000e+01,111.0,14.00000,141.000000,141.000000,168.0,37.0,77.000000,17.0,10.0,222.000000,21.0,6.0,25.000000,8.0,311.000000,300.000000,311.000000,41.000000,4.0,13.0,11.0,22.000000,22.000000,6.000000,9.000000,11.0,6.0,3.0,27.0,21.0,9.0,4.0,4.0,5.0,2.0,2.0,2.0,16.0,1.0,8.0,2.0,8.000000,8.00000,4.0,2.0,2.0,11.0,28.0,4.0,4.0,1.0,5.0,4.0,1.0,1.0,4.0,2.0,13.0,1.0,1.0,2.0,6.0,1.0,2.0,4.0,2.0,2.0,2.0,2.0,4.0,4.0,2.0,3.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,3.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0
mean,2.696500e+09,447.341629,12.575949,1.552108,46.558926,29.083145,0.793585,44.176722,47.860262,8.226562,11650.351351,51.181067,51.101565,17.539149,3.109729,454.244004,40.739262,0.0,0.0,545.655210,210.726562,49.626795,22.699292,92.911765,49.376287,98.071105,49.903063,1.0,14.528354,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,423.535714,7.802201e+08,1.0,395.00000,51.404255,50.307092,0.0,0.0,4.662338,0.0,0.0,0.333333,0.0,0.0,2.640000,0.0,7.479100,6.443333,2.723473,484.536585,0.0,0.0,0.0,11602.136364,449.545455,11.666667,591.333333,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,592.875000,478111.37500,0.0,0.0,0.0,243.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,243.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,243.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,2.727314e+07,265.876364,20.844422,0.922337,28.431365,17.428497,0.404735,26.398014,29.701580,4.878387,7449.230488,22.768123,27.578575,10.922255,1.991682,265.294235,30.825532,0.0,0

### Visualizaciones iniciales

In [25]:
# summary = utils.comprehensive_data_exploration(df)

## TRATAMIENTO DE DATOS

### Limpieza de datos

In [26]:
df_clean = utils.clean_data(df)

In [28]:
df_stats_per_season = utils.agregate_players_stats(df_clean)

In [29]:
df_stats_per_match = utils.agregate_players_stats_per_match(df_clean)